In [80]:
import diffrax
import jax
import ticktack
import matplotlib.pyplot as pyplot

In [94]:
model = ticktack.load_presaved_model("Guttler15", production_rate_units="atoms/cm^2/s")
model.compile()

STEADY_PROD = model.equilibrate(target_C_14=707.)
STEADY_STATE = model.equilibrate(production_rate=STEADY_PROD)
# STEADY_PROD *= 3.747273140033743
PROD_COEFFS = model._production_coefficients
MATRIX = model._matrix

del model

In [95]:
@jax.tree_util.Partial
@jax.jit
def production(t, args):
    start_time, duration, phase, area = jax.numpy.array(args)
    middle = start_time + duration / 2.
    height = area / duration

    gauss = height * jax.numpy.exp(- ((t - middle) / (1. / 1.93516 * duration)) ** 16.)
    sine = 1.8803862513018528 + 0.18 * 1.8803862513018528 *\
        jax.numpy.sin(2 * jax.numpy.pi / 11 * t + phase * 2 * jax.numpy.pi / 11) 
        
    return (sine + gauss) * 3.747273140033743

In [96]:
@jax.tree_util.Partial
@jax.jit
def derivative(t, y, args, /, matrix=MATRIX, production=production, prod_coeffs=PROD_COEFFS): 
    
    ans = jax.numpy.matmul(matrix, y) 
    production_rate_constant = production(t, args)
    production_term = prod_coeffs * production_rate_constant 
    return ans + production_term 

In [97]:
@jax.tree_util.Partial
@jax.jit
def solve(y_initial, time, args, /, solver=diffrax.Dopri5(), dydx=derivative):
    term = diffrax.ODETerm(dydx)
    saveat = diffrax.SaveAt(ts=time)
    stepsize = diffrax.PIDController(rtol=1e-5, atol=1e-5)

    t0 = time.min()
    t1 = time.max()
    dt0 = time[1] - time[0] / 1000
    
    sol = diffrax.diffeqsolve(args=args, terms=term, solver=solver, t0=t0, t1=t1,\
        dt0=dt0, y0=y_initial, saveat=saveat, stepsize_controller=stepsize)
    
    return sol.ys[:, 1]

In [98]:
with open("miyake12.csv") as data:
    _ = next(data)  # String titles 
    data = jax.numpy.array([row.strip().split(" ") for row in data],\
        dtype=jax.numpy.float64)
    data = data.T

In [124]:
@jax.tree_util.Partial
@jax.jit
def log_likelihood(args, /, data=data, func=solve, y0=STEADY_STATE):
    solution = func(y0, data[0], args)
    solution = (solution - STEADY_STATE[1]) / STEADY_STATE[1]
    solution += jax.numpy.mean(data[1][:4])
    chi_squared = (solution - data[1]) ** 2 / data[2] ** 2
    return - 0.5 * jax.numpy.sum(chi_squared)
    

In [106]:
parameters = (774.86, 0.25, 0.8, 6.44)

In [127]:
%%timeit
log_likelihood(parameters)

476 µs ± 49.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [132]:
jac = jax.jit(jax.grad(log_likelihood))

In [135]:
%%timeit
jac(parameters)

4.5 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [136]:
hes = jax.jit(jax.hessian(log_likelihood))

In [139]:
%%timeit
hes(parameters)

44.7 ms ± 2.55 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
